In [56]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from datetime import date, timedelta
import datetime
from urllib.request import Request, urlopen

In [57]:
# Crawler for www.findaphd.com

df = pd.DataFrame(columns=["title","country","university","deadline","lastUpdate","link"])

pageNo = 1
while True:
    print(f"getting results for page {pageNo}")

    response = requests.get(f"https://www.findaphd.com/phds/engineering/?10M7o0&Show=M&PG={pageNo}")

    soup = BeautifulSoup(response.content, "html.parser")
    elements = soup.find("div", attrs={"id":"main"})
    elements = elements.findChild("div", attrs={"id":"main-col"})
    elements = elements.findChild("div", attrs={"id":"SearchResults"})
    elements = elements.findChildren("div", attrs={"id":re.compile('searchResultImpression\d+')}) if elements.findChildren("div", attrs={"id":re.compile('searchResultImpression\d+')}) else []

    if 0 < len(elements):
        i = df.shape[0]
        for e in elements:
            title = e.find("a", attrs={"class":"h4"})
            country = e.find("a", attrs={"class":"instLink"})
            university = e.find("span", attrs={"class":"phd-result__dept-inst--title"})
            deadline = e.find("div", attrs={"class":"phd-icon-area"})
            link = e.find("a", attrs={"class":"btn"})
            lastUpdate = e.find("div", attrs={"class":"apply"})

            # Handling the values that were not found
            title = title.text if title != None else "NaN"
            country = country.find("img")["title"] if country != None else "NaN"
            university = university.text if university != None else "NaN"
            if deadline != None:
                if len(deadline.findChildren("a", attrs={"class":"hoverTitle"})) != 0:
                    deadline = deadline.findChildren("a", attrs={"class":"hoverTitle"})[0].text
                else:
                    deadline = "NaN"
            else:
                deadline = "NaN"
            if e.find("a", attrs={"class":"btn"}) != None:
                link = "http://www.findaphd.com"+link["href"]
            else:
                link = "NaN"
            lastUpdate = lastUpdate.text.replace("Updated: ","").replace(" ","-").replace("\n","").replace("\xa0","") if lastUpdate != None else "NaN"

            df.loc[i] = [title, country, university, deadline, lastUpdate, link]
            i+=1
    else:
        break
    
    time.sleep(1)
    pageNo += 1

s_ = datetime.datetime.now().date().strftime("%d-%b-%Y")
df.to_excel(f"find_a_phd-{s_}.xlsx")

getting results for page 1
getting results for page 2
getting results for page 3
getting results for page 4
getting results for page 5
getting results for page 6
getting results for page 7
getting results for page 8
getting results for page 9
getting results for page 10
getting results for page 11
getting results for page 12
getting results for page 13
getting results for page 14
getting results for page 15
getting results for page 16
getting results for page 17
getting results for page 18
getting results for page 19
getting results for page 20
getting results for page 21
getting results for page 22
getting results for page 23
getting results for page 24
getting results for page 25
getting results for page 26
getting results for page 27
getting results for page 28
getting results for page 29


In [58]:
# Crawler for www.academictransfer.com

df = pd.DataFrame(columns=["title","country","university","deadline","posted","link"])

pageNo = 1
while True:
    print(f"getting results for page {pageNo}")

    response = requests.get(f"https://www.academictransfer.com/en/jobs/?vacancy_type=scientific&q=&function_types=1&scientific_fields=3&order=&page={pageNo}")

    soup = BeautifulSoup(response.content, "html.parser")
    elements = soup.findChild("div", attrs={"class":re.compile('_search_results_\w+')})
    
    if elements != None:
        elements = elements.findChildren("div", attrs={"id":re.compile('v\d+')})
        if 0 < len(elements):
            i = df.shape[0]
            for e in elements:
                link = e.findChild("a")
                if link != None:
                    link = "https://academictransfer.com/"+link["href"]
                else:
                    link = "NaN"
                
                country = "-"

                e = e.findChild("div",attrs={"class":re.compile("_VacancyInfo_VacancyInfo_\w+")})
                title = e.findChild("h6")
                if title != None: 
                    title = title.text.strip()
                else:
                    title = "NaN"
                
                e = e.findChild("div",attrs={"class":re.compile("_Metadata_Metadata_-\w+")})
                university = e.findChild("div",attrs={"class":re.compile("_Metadata_favorite_wrapper_\w+")})
                if university != None: 
                    university = university.text.strip()
                else:
                    university = "NaN"
                    
                deadline = e.findChild("div",attrs={"class":re.compile("_Metadata_deadline_\w+")})
                if deadline != None: 
                    if "today" in deadline.text.strip().lower():
                        deadline = datetime.datetime.today().strftime("%d-%b-%Y")
                    elif "yesterday" in deadline.text.strip().lower():
                        deadline = (datetime.datetime.today()-timedelta(1)).strftime("%d-%b-%Y")
                    else:
                        deadline = deadline.text.strip() + " 2024"
                        deadline = datetime.datetime.strptime(deadline, "%d %b %Y").strftime("%d-%b-%Y")
                else:
                    deadline = "NaN"
 
                posted = e.findChild("div",attrs={"class":re.compile("_Metadata_created_\w+")})
                if posted != None: 
                    if "today" in posted.text.strip().lower():
                        posted = datetime.datetime.today().strftime("%d-%b-%Y")
                    elif "yesterday" in posted.text.strip().lower():
                        posted = (datetime.datetime.today()-timedelta(1)).strftime("%d-%b-%Y")
                    else:
                        posted = posted.text.strip() + " 2024"
                        posted = datetime.datetime.strptime(posted, "%d %b %Y").strftime("%d-%b-%Y")
                else:
                    posted = "NaN"

                df.loc[i] = [title, country, university, deadline, posted, link]
                i+=1
            pageNo+= 1
            time.sleep(.5)
        else:
            break
    else:
        break

_s = datetime.datetime.now().date().strftime("%d-%b-%Y")
df.to_excel(f"academic_positions-{_s}.xlsx")

getting results for page 1
getting results for page 2
getting results for page 3
getting results for page 4
getting results for page 5
getting results for page 6
getting results for page 7
getting results for page 8
getting results for page 9
getting results for page 10
getting results for page 11
getting results for page 12


In [59]:
# Crawler for www.academicpositions.com

df = pd.DataFrame(columns=["title","country","university","deadline","lastUpdate","link"])

pageNo = 1
while True:
    print(f"getting results for page {pageNo}")

    response = requests.get(f"https://academicpositions.com/jobs/position/phd/field/engineering?sort=closing&page={pageNo}")

    soup = BeautifulSoup(response.content, "html.parser")
    elements = soup.find("div", attrs={"id":"list-jobs"})
    
    if elements != None:
        elements = elements.findChildren("div", attrs={"class":"job-list-item"})

        if 0 < len(elements):
            i = df.shape[0]
            for e in elements:

                title = e.findChild("h4")
                if title != None: 
                    title = title.text.strip()
                else:
                    title = "NaN"
                    
                country = e.findChild("div", attrs={"class":"job-locations"})
                if country != None: 
                    country = country.text.split(",")[-1].strip()
                else:
                    country = "NaN"

                university = e.findChild("a", attrs={"class":"job-link"})
                if university != None: 
                    university = university.text.strip()
                else:
                    university = "NaN"
                    
                foundDeadLine = False
                deadline = e.findChildren("div", attrs={"class":"text-muted"})
                if deadline != None:
                    foundDeadLine = False
                    for item in deadline:
                        if "Closing in:" in item.text:
                            deadline = item.findChildren("div", attrs={"class":"col-auto"})
                            for item in deadline:
                                if "Closing in:" in item.text:
                                    deadline = item.text.strip().replace("Closing in:","").strip()

                                    if "hours" in deadline:
                                        deadline = (datetime.datetime.now().date() + timedelta(1)).strftime("%d-%b-%Y")
                                        foundDeadLine = True
                                    elif "days" in deadline:
                                        _t = int(deadline.replace(" days",""))
                                        deadline = (datetime.datetime.now().date() + timedelta(_t)).strftime("%d-%b-%Y")
                                        foundDeadLine = True
                                    elif "-" in deadline:
                                        _t = deadline.replace(" days","")
                                        deadline = datetime.datetime.strptime(_t, "%Y-%m-%d").strftime("%d-%b-%Y")
                                        foundDeadLine = True
                                    else:
                                        deadline = "NaN"
                                    # print(deadline)
                                    foundDeadLine = True

                                if not foundDeadLine:
                                    deadline = "NaN"
                                    
                            if not foundDeadLine:
                                deadline = "NaN"
                        if not foundDeadLine:
                            deadline = "NaN"
                else:
                    deadline = "NaN"
                
                link = e.findChildren("a", attrs={"class":"job-link"})
                if 1< len(link):
                    link = "https://academicpositions.com/"+link[1]["href"]
                else:
                    link = "NaN"
                    
                df.loc[i] = [title, country, university, deadline, "NaN", link]

                i+=1
            pageNo+= 1
            time.sleep(.5)
        else:
            break
    else:
        break
s_ = datetime.datetime.now().date().strftime("%d-%b-%Y")
df.to_excel(f"academic_positions-{s_}.xlsx")

getting results for page 1
getting results for page 2
getting results for page 3
getting results for page 4
getting results for page 5
getting results for page 6
getting results for page 7
getting results for page 8
getting results for page 9
getting results for page 10


In [60]:
# Crawler for www.euraxess.ec.europa.eu
df = pd.DataFrame(columns=["title","country","university","deadline","publishDate","link"])

pageNo = 1
while True:
    print(f"getting results for page {pageNo}")

    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(f"https://euraxess.ec.europa.eu/jobs/search?f%5B0%5D=positions%3Aphp_positions&page={pageNo}",headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    elements = (soup.findChildren("div", attrs={"id":"job-teaser-content"}))

    if 0 < len(elements):
        i = df.shape[0]
        for e in elements:
            title = e.findChild("article")
            title = title.findChild("h1")
            if title != None:
                title = title.text.strip()
            else:
                title = "NaN"
            
            country = e.findChild("div", attrs={"class":"label-thumbnail-wrapper"})
            country = country.findChildren("li")
            if 1 < len(country):
                country = country[1].text.strip()
            else:
                country = "NaN"

            university = e.findChild("article")
            university = university.findChild("ul", attrs = {"class":"ecl-content-block__primary-meta-container"})
            if university != None:
                university = university.findChildren("li")
                if len(university) != 0:
                    university = university[0].text
                else:
                    university = "NaN"
            else:
                university = "NaN"

            publishDate = e.findChild("article")
            publishDate = publishDate.findChild("ul", attrs = {"class":"ecl-content-block__primary-meta-container"})
            if publishDate != None:
                publishDate = publishDate.findChildren("li")
                if len(publishDate) != 0:
                    publishDate = publishDate[1].text.replace("Posted on:","").strip()
                    publishDate = datetime.datetime.strptime(publishDate, "%d %B %Y").strftime("%d-%b-%Y")
                else:
                    publishDate = "NaN"
            else:
                publishDate = "NaN"
                
            deadline = e.findChild("article").findChild("div", attrs={"class","id-Application-Deadline"})
            if deadline != None:
                deadline = deadline.text.replace("Application Deadline:","").strip().split("-")[0].strip()
                deadline = datetime.datetime.strptime(deadline,"%d %b %Y").strftime("%d-%b-%Y")
            else:
                deadline = "NaN"
            
            link = e.findChild("article")
            link = link.findChild("h1")
            if link != None:
                link = link.findChild("a")
                if link != None:
                    link = "https://euraxess.ec.europa.eu"+link["href"]
                else:
                    link = "NaN"
            else:
                link = "NaN"
            
            df.loc[i] = [title, country, university, deadline, publishDate, link]

            i+=1
    
        time.sleep(0.1)
        pageNo += 1
    else:
        break

s_ = datetime.datetime.now().date().strftime("%d-%b-%Y")
df.to_excel(f"euraxess-{s_}.xlsx")

getting results for page 1
getting results for page 2
getting results for page 3
getting results for page 4
getting results for page 5
getting results for page 6
getting results for page 7
getting results for page 8
getting results for page 9
getting results for page 10
getting results for page 11
getting results for page 12
getting results for page 13
getting results for page 14
getting results for page 15
getting results for page 16
getting results for page 17
getting results for page 18
getting results for page 19
getting results for page 20
getting results for page 21
getting results for page 22
getting results for page 23
getting results for page 24
getting results for page 25
getting results for page 26
getting results for page 27
getting results for page 28
getting results for page 29
getting results for page 30
getting results for page 31
getting results for page 32
getting results for page 33
getting results for page 34
getting results for page 35
getting results for page 36
g